## Prompt Caching

*(Coding along with deeplearning.ai's online course [Building toward Computer Use with Anthropic - Learn how an AI Assistant is built to use and accomplish tasks on computers](https://learn.deeplearning.ai/courses/building-toward-computer-use-with-anthropic/lesson/1/introduction) taught by Colt Steele)*

Prompt caching is a technique in large language model (LLM) applications where previously used prompts and their responses are stored and reused, rather than making a new LLM API call for identical or similar prompts. This reduces API costs, improves response times, and decreases computational load. When a new prompt comes in, the system first checks if a matching or similar prompt exists in the cache before sending the request to the LLM.

In [1]:
# https://learn.deeplearning.ai/courses/building-toward-computer-use-with-anthropic/lesson/5/real-world-prompting
from anthropic import Anthropic
import pandas as pd

anthropic_api_key = pd.read_csv("~/tmp/anthropic/anthropic-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME="claude-3-5-sonnet-20241022"

Don't be a fool and sent your api key to github


#### __Loading The Book__

In [2]:
with open('../assets/files/frankenstein.txt', 'r') as file:
    book_content = file.read()

In [3]:
len(book_content)

438804

In [4]:
book_content[1000:2000]

'by Mary Wollstonecraft (Godwin) Shelley\n\n\n CONTENTS\n\n Letter 1\n Letter 2\n Letter 3\n Letter 4\n Chapter 1\n Chapter 2\n Chapter 3\n Chapter 4\n Chapter 5\n Chapter 6\n Chapter 7\n Chapter 8\n Chapter 9\n Chapter 10\n Chapter 11\n Chapter 12\n Chapter 13\n Chapter 14\n Chapter 15\n Chapter 16\n Chapter 17\n Chapter 18\n Chapter 19\n Chapter 20\n Chapter 21\n Chapter 22\n Chapter 23\n Chapter 24\n\n\n\n\nLetter 1\n\n_To Mrs. Saville, England._\n\n\nSt. Petersburgh, Dec. 11th, 17—.\n\n\nYou will rejoice to hear that no disaster has accompanied the\ncommencement of an enterprise which you have regarded with such evil\nforebodings. I arrived here yesterday, and my first task is to assure\nmy dear sister of my welfare and increasing confidence in the success\nof my undertaking.\n\nI am already far north of London, and as I walk in the streets of\nPetersburgh, I feel a cold northern breeze play upon my cheeks, which\nbraces my nerves and fills me with delight. Do you understand this\n

### Uncached Request

In [5]:
import time
def make_non_cached_api_call():
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    # marking the bounds of the book with xml
                    "text": "<book>" + book_content + "</book>"
                },
                {
                    # our actual question in a second message block
                    "type": "text",
                    "text": "What happens in chapter 3?"
                }
            ]
        }
    ]

    start_time = time.time() # time request gets send
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=500,
        messages=messages,
    )
    end_time = time.time() # time response is received 

    return response, end_time - start_time # returning response plus delta of time

In [6]:
non_cached_response, non_cached_time = make_non_cached_api_call()
print(f"Non-cached time: {non_cached_time:.2f} seconds") # time it took to get the response

print("\nOutput (non-cached):") 
print(non_cached_response.content) # the non cached response

Non-cached time: 98.37 seconds

Output (non-cached):
[TextBlock(citations=None, text='In Chapter 3 of Frankenstein, several key events occur:\n\n1. Victor Frankenstein begins his studies at the University of Ingolstadt at age 17.\n\n2. He meets his first professor, M. Krempe, who harshly criticizes Victor\'s previous studies of alchemists like Cornelius Agrippa, dismissing them as outdated and worthless.\n\n3. Victor then meets Professor M. Waldman, who has a much more encouraging and gentle approach. Unlike Krempe, Waldman acknowledges the contributions of the ancient alchemists while emphasizing the superiority of modern science.\n\n4. Waldman\'s lecture inspires Victor, particularly when he speaks about how modern scientists can "command the thunders of heaven, mimic the earthquake, and even mock the invisible world with its own shadows."\n\n5. Victor becomes deeply devoted to his studies, particularly chemistry and other branches of natural philosophy. He excels in his work and mak

[TextBlock(citations=None, text='In Chapter 3 of Frankenstein, several key events occur:\n\n1. Victor Frankenstein begins his studies at the University of Ingolstadt at age 17.\n\n2. He meets his first professor, M. Krempe, who harshly criticizes Victor\'s previous studies of alchemists like Cornelius Agrippa, dismissing them as outdated and worthless.\n\n3. Victor then meets Professor M. Waldman, who has a much more encouraging and gentle approach. Unlike Krempe, Waldman acknowledges the contributions of the ancient alchemists while emphasizing the superiority of modern science.\n\n4. Waldman\'s lecture inspires Victor, particularly when he speaks about how modern scientists can "command the thunders of heaven, mimic the earthquake, and even mock the invisible world with its own shadows."\n\n5. Victor becomes deeply devoted to his studies, particularly chemistry and other branches of natural philosophy. He excels in his work and makes rapid progress, even making discoveries that impre